### Feature Engineering and Data Preprocessing

In [1]:
import os
import sys

system_path = os.path.abspath('..')
if system_path not in sys.path:
    sys.path.append(system_path)

%load_ext autoreload
%autoreload 2

In [22]:
from src.data_processing import FeatureEngineer, load_data, save_processed_data,get_full_preprocessing_pipeline

# Load the data
data = load_data('../data/raw/data.csv')


Successfully loaded data from ../data/raw/data.csv. Shape: (95662, 16)


In [23]:
data.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

In [24]:
import sidetable
data.stb.freq(['PricingStrategy'])

,PricingStrategy,count,percent,cumulative_count,cumulative_percent
0,2,79848,83.468880,79848,83.468880
1,4,13562,14.176998,93410,97.645878
2,1,1867,1.951663,95277,99.597541
3,0,385,0.402459,95662,100.000000


In [25]:
data.stb.freq(['FraudResult'])

,FraudResult,count,percent,cumulative_count,cumulative_percent
0,0,95469,99.798248,95469,99.798248
1,1,193,0.201752,95662,100.000000


In [26]:
data.stb.missing()

,missing,total,percent
TransactionId,0,95662,0.0
BatchId,0,95662,0.0
AccountId,0,95662,0.0
SubscriptionId,0,95662,0.0
CustomerId,0,95662,0.0
CurrencyCode,0,95662,0.0
CountryCode,0,95662,0.0
ProviderId,0,95662,0.0
ProductId,0,95662,0.0
ProductCategory,0,95662,0.0


In [29]:
data.stb.freq(['ProviderId'])

,ProviderId,count,percent,cumulative_count,cumulative_percent
0,ProviderId_4,38189,39.920763,38189,39.920763
1,ProviderId_6,34186,35.736238,72375,75.657001
2,ProviderId_5,14542,15.201438,86917,90.858439
3,ProviderId_1,5643,5.898894,92560,96.757333
4,ProviderId_3,3084,3.223851,95644,99.981184
5,ProviderId_2,18,0.018816,95662,100.000000


In [30]:
import pandas as pd
full_preprocessing_pipeline = get_full_preprocessing_pipeline()

# Fit and transform the raw data using the full pipeline
# The output `processed_data_array` will be a NumPy array
# containing the scaled numerical features and one-hot encoded categorical features.
processed_data_array = full_preprocessing_pipeline.fit_transform(data)
print(f"Shape of fully processed data (NumPy array): {processed_data_array.shape}")
# To get the column names for the processed data (useful for inspection/debugging)
# This requires accessing the fitted transformers within the pipeline
try:
    # Get the preprocessor step from the full pipeline
    preprocessor_step = full_preprocessing_pipeline.named_steps['preprocessor']
    # Get the feature names for numerical features (they remain the same)
    numerical_feature_names = preprocessor_step.transformers_[0][2]
    # Get the feature names for one-hot encoded categorical features
    onehot_encoder = preprocessor_step.named_transformers_['cat'].named_steps['onehot']
    categorical_feature_names = onehot_encoder.get_feature_names_out(
        preprocessor_step.transformers_[1][2] # Pass the original categorical column names
    )
    # Combine all feature names
    all_processed_feature_names = list(numerical_feature_names) + list(categorical_feature_names)
    # Convert the processed NumPy array back to a DataFrame for saving/inspection
    processed_df = pd.DataFrame(processed_data_array, columns=all_processed_feature_names)
    print("\nFirst 5 rows of fully processed (model-ready) data:")
    print(processed_df.head())
    print(f"Fully processed data shape: {processed_df.shape}")
    # Save the fully processed data
    save_processed_data(processed_df, "../data/processed/Preprocessed_data.csv")
except Exception as e:
    print(f"Could not reconstruct DataFrame with column names after full preprocessing: {e}")
    print("The raw NumPy array of processed features is available in 'processed_data_array'.")

Shape of fully processed data (NumPy array): (3742, 39)

First 5 rows of fully processed (model-ready) data:
   total_transaction_amount  ...  mode_PricingStrategy_4
0                 -0.022752  ...                     0.0
1                 -0.062842  ...                     0.0
2                  0.020976  ...                     0.0
3                 -0.062474  ...                     0.0
4                 -0.028833  ...                     0.0

[5 rows x 39 columns]
Fully processed data shape: (3742, 39)
Processed data saved to ../data/processed/Preprocessed_data.csv


In [35]:
processed_df.columns

Index(['total_transaction_amount', 'avg_transaction_amount',
       'transaction_count', 'std_transaction_amount', 'total_value',
       'avg_value', 'fraud_transaction_count', 'fraud_rate',
       'unique_CurrencyCode_count', 'unique_CountryCode_count',
       'unique_ProviderId_count', 'unique_ProductCategory_count',
       'unique_ChannelId_count', 'unique_PricingStrategy_count',
       'mode_CurrencyCode_UGX', 'mode_CountryCode_256',
       'mode_ProviderId_ProviderId_1', 'mode_ProviderId_ProviderId_2',
       'mode_ProviderId_ProviderId_3', 'mode_ProviderId_ProviderId_4',
       'mode_ProviderId_ProviderId_5', 'mode_ProviderId_ProviderId_6',
       'mode_ProductCategory_airtime', 'mode_ProductCategory_data_bundles',
       'mode_ProductCategory_financial_services',
       'mode_ProductCategory_movies', 'mode_ProductCategory_other',
       'mode_ProductCategory_ticket', 'mode_ProductCategory_transport',
       'mode_ProductCategory_tv', 'mode_ProductCategory_utility_bill',
       '

In [31]:
processed_df

,total_transaction_amount,avg_transaction_amount,transaction_count,std_transaction_amount,total_value,avg_value,fraud_transaction_count,fraud_rate,unique_CurrencyCode_count,unique_CountryCode_count,unique_ProviderId_count,unique_ProductCategory_count,unique_ChannelId_count,unique_PricingStrategy_count,mode_CurrencyCode_UGX,mode_CountryCode_256,mode_ProviderId_ProviderId_1,mode_ProviderId_ProviderId_2,mode_ProviderId_ProviderId_3,mode_ProviderId_ProviderId_4,mode_ProviderId_ProviderId_5,mode_ProviderId_ProviderId_6,mode_ProductCategory_airtime,mode_ProductCategory_data_bundles,mode_ProductCategory_financial_services,mode_ProductCategory_movies,mode_ProductCategory_other,mode_ProductCategory_ticket,mode_ProductCategory_transport,mode_ProductCategory_tv,mode_ProductCategory_utility_bill,mode_ChannelId_ChannelId_1,mode_ChannelId_ChannelId_2,mode_ChannelId_ChannelId_3,mode_ChannelId_ChannelId_5,mode_PricingStrategy_0,mode_PricingStrategy_1,mode_PricingStrategy_2,mode_PricingStrategy_4
0,-0.022752,-0.088217,0.964082,-0.109030,-0.018139,-0.101792,-0.066617,-0.086096,0.0,0.0,2.167925,0.924070,2.304899,4.184641,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,-0.062842,-0.090744,-0.243141,-0.140432,-0.092838,-0.109037,-0.066617,-0.086096,0.0,0.0,-1.382737,-1.153977,-1.404749,-0.756642,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.020976,-0.057828,0.128312,0.036788,0.012344,-0.066973,-0.066617,-0.086096,0.0,0.0,1.280259,3.002117,2.304899,0.890452,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,-0.062474,-0.081798,-0.253459,-0.140432,-0.092470,-0.100078,-0.066617,-0.086096,0.0,0.0,-1.382737,-1.153977,-1.404749,-0.756642,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,-0.028833,-0.074518,0.035449,-0.083724,-0.045407,-0.085291,-0.066617,-0.086096,0.0,0.0,0.392594,-0.114953,0.450075,0.890452,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3737,-0.062849,-0.092265,-0.222505,-0.130531,-0.092094,-0.107514,-0.066617,-0.086096,0.0,0.0,0.392594,-0.114953,0.450075,-0.756642,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3738,-0.054008,-0.019177,-0.243141,-0.103939,-0.084000,-0.037365,-0.066617,-0.086096,0.0,0.0,-1.382737,-1.153977,-1.404749,-0.756642,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3739,-0.060829,-0.084082,-0.222505,-0.105116,-0.089991,-0.098985,-0.066617,-0.086096,0.0,0.0,0.392594,0.924070,0.450075,-0.756642,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3740,-0.061370,-0.063906,-0.253459,-0.140432,-0.091365,-0.082160,-0.066617,-0.086096,0.0,0.0,-1.382737,-1.153977,-1.404749,-0.756642,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [34]:
processed_df.stb.freq(['mode_PricingStrategy_0'])

,mode_PricingStrategy_0,count,percent,cumulative_count,cumulative_percent
0,0.0,3713,99.225013,3713,99.225013
1,1.0,29,0.774987,3742,100.000000
